In [ ]:
import tensorflow as tf
tf.keras.backend.set_floatx('float32')
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True

import matplotlib.pyplot as plt
import numpy as np

def printValues(array, scale="prob", sample=None):
    n = 2 if type(sample) is np.ndarray else 1
    fig, axs = plt.subplots(1, n, figsize=(6*n, 6))
    x = np.arange(0, 10)
    ax = axs if n == 1 else axs[0]
    ax.bar(x, array[0])
    if scale != "logit":
        ax.set(ylim=[0, 1])
    ax.set_xticks(x)
    if n == 2:
        axs[1].matshow(sample.reshape((28,28)))

# Tensorflow und Keras

[Keras](https://keras.io) ist eine Open-Source-Bibliothek für Deep Learning, die in Python geschrieben wurde.
Keras unterstützt verschiedene Machine Learning Backends, unter anderem auch [Tensorflow](https://www.tensorflow.org/). Der Vorteil von Keras ist, ist dass die API sehr sauber und intuitiv ist.

Ab 2017 hat Google begonnen die Keras-API in Tensorflow zu integrieren, daher kommen die Module im Namespace `tf.keras`.

Seit 2019 werden andere Backends als Tensorflow (sprich Theano oder CNTK) von Keras nicht mehr unterstützt.

Seit Dezember 2019 empfiehlt das Keras Team auf Tensorflow 2 mit tf.keras zu setzen.

# Klassifikation der MNIST Ziffern

Wir laden uns die 60.000 Trainings- und 10.000 Test-Bilder im Format 28x28.

**Beachte:** Die Daten werden einmalig heruntergeladen und lokal persistiert - das erste Mal kann abhängig von Bandbreite etwas länger dauern.

In [ ]:
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_info

## Daten vorbereiten

Ein TensorFlow **`tf.data.Dataset`** dient der Verwaltung von potentiell gewaltigen Datenmengen.

Die `tf.data.Dataset` API unterstützt das Schreiben sprechender und effizienter Eingabe-Pipelines. Die Dataset-Verwendung folgt einem gemeinsamen Muster:

-  Quelldatensatz aus den Eingabedaten erstellen
-  Ggf. Dataset-Transformationen zur Vorverarbeitung der Daten anwenden
-  Generator über den Datensatz erzeugen
-  Iterieren mittels des Generators

Die Iteration erfolgt in einem Streaming-Verfahren, so dass der vollständige Datensatz nicht in den Speicher passen muss.

**Einige Methoden:**
- `repeat(count)`: Wiederholt diesen Datensatz, so dass jeder Originalwert `count`-mal gesehen wird. Default: unendlich
- `shuffle(buffer_size)`: Mischt die Elemente dieses Datensatzes nach dem Zufallsprinzip um.

    Dieser Datensatz füllt einen Puffer mit buffer_size-Elementen, nimmt dann nach dem Zufallsprinzip Stichproben von Elementen aus diesem Puffer und ersetzt die ausgewählten Elemente durch neue Elemente. Für ein perfektes Mischen ist eine Puffergröße erforderlich, die größer oder gleich der vollen Größe des Datensatzes ist.

    Wenn Ihr Datensatz beispielsweise 10.000 Elemente enthält, buffer_size jedoch auf 1.000 eingestellt ist, wählt die Zufallsmischung zunächst ein Zufallselement aus nur den ersten 1.000 Elementen im Puffer aus. Sobald ein Element ausgewählt ist, wird sein Platz im Puffer durch das nächste (d. h. das 1.001-st.) Element ersetzt, wobei der Puffer mit 1.000 Elementen beibehalten wird.

- `batch(batch_size)`: Sammelt aufeinander folgende Elemente dieses Datensatzes zu Batch-Stapeln.
- `prefetch(buffer_size)`: Erstellt ein Dataset, das Elemente aus diesem Dataset vorzeitig abruft.

    Die meisten Dataset-Eingabe-Pipelines sollten mit einem Aufruf von `prefetch` enden. Auf diese Weise können spätere Elemente vorbereitet werden, während das aktuelle Element verarbeitet wird. Dies verbessert oft die Latenzzeit und den Durchsatz auf Kosten der Verwendung von zusätzlichem Speicher zum Speichern vorgeladener Elemente.

Die Grauwerte müssen wir zunächst von [0 ... 255] zu [0.0 ... 1.0] konvertieren.

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train_scaled = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test_scaled = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Als nächstes erzeugen wir unseren Datengenerator, über den das Training nachher iteriert:

In [ ]:
train = ds_train_scaled.shuffle(1024, reshuffle_each_iteration=True).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

Ebenso erzeugen wir unseren Test-Generator, allerdings hier ohne Shufflen.

In [ ]:
test = ds_test_scaled.batch(64).prefetch(tf.data.experimental.AUTOTUNE)

# Modell vorbereiten

Ein Keras-Modell hat einen Lebenszyklus, und dieses Wissen bildet das Rückgrat für das Verständnis der tf.keras-API.

Die fünf Schritte im Lebenszyklus sind wie folgt:

-  Definieren:
   Netzarchitektur entwerfen
-  Kompilieren: Interne Datenstrukturen werden für die effiziente Verwendung vorbereitet
-  Fitten: Der rechenintensive Teil: das Trainieren des Modells
-  Evaluieren: Validierung des Modells gegen das Dev-Set, Optimierung der Hyperparameter, Test
-  Verwenden


## Definition des neuronalen Netzes

Für unser erstes Modell verwenden wir die "Sequential API" - dies ist die einfachste und intuitivste Methode: unser Modell konstruieren wir aus einer Sequenz von Layern.

Im **Inputlayer** wandeln wir die 28x28 Pixel in einen 784-Vektor um.

Der **erste hidden Layer** ist ein voll verbundener Layer mit 128 Nodes und ReLU-Aktivierung. "input_shape" muss nur beim Input-Layer angegeben werden, danach weiß Tensorflow ja selbst, was die Ausgabe des vorigen und damit die Eingabe des Folgelayers ist.

Der **zweite hidden Layer** ist ein **Dropout**-Layer. Der Parameter 0.2 besagt, dass das Netz in jeder Iteration zufällig 20% der Weights vergisst. Dies ist eine Form der Regularisierung (reduziert das Auswendiglernen von Trainingsdaten).

Der **Output-Layer** schließlich ist ein voll verbundener Layer mit 10 Nodes für unsere 10 Klassen. Eine Aktivierungsfunktion ist an diesem Layer nicht angegeben, daher wird die Default-Aktivierung verwendet, das ist schlicht die Identity-Funktion $f(x)=x$.

D.h. unser Outputlayer bildet einfach eine gewichtete Summen über die 128 Nodes des vorherigen Layers.

Mit den Bezeichnungen unserer Folien ergibt sich daher (wenn wir die Ausgabe des Flatten-Layer als Inputlayer betrachten):

$$a^{[1]}=ReLU({w^{[1]}}^Tx+b^{[1]})$$
$$a^{[2]}=Dropout(a^{[1]})$$
$$a^{[3]}={w^{[3]}}^Ta^{[2]}+b^{[3]}=\hat{y}$$

mit $x \in \mathbb{R}^{128}$ und $\hat{y} \in \mathbb{R}^{10}$.

**Dropout** führt dazu, dass 20% der $a^{[1]}$ auf 0 gesetzt und die anderen proportional erhöht werden, so dass $\sum a^{[1]}=\sum a^{[2]}$ gilt.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
print(model.summary())
print(f'Anzahl Parameters Layer Dense 128: #a * #w + #b = 784 * 128 + 128 = {784 * 128 + 128}')
print(f'Anzahl Parameters Layer Dense 10:  #a * #w + #b = 128 * 10 + 10 = {128 * 10 + 10}')

In [ ]:
samples = ds_train_scaled.as_numpy_iterator()
[sample, label] = next(samples)
sample.shape, label.shape

Für jedes Sample gibt das Modell einen Vektor von "Logits" oder "Log-Odds" zurück, einen für jede Klasse.

Tensorflow rechnet mit sogenannten **"Tensoren"**. Dies sind schlicht Datenstrukturen, die sowohl Skalare, als auch Vektoren und multidimensionale Matrizen abbilden können. Die Dimension eines Tensors heißt "Rank". D.h. ein Skalar ist ein Tensor von Rank=0, ein Vektor hat Rank=1, u.s.w.

Wenn wir uns den Wert eines Tensors anschauen wollen, müssen wir die Methode `.numpy()` auf dem Tensor aufrufen.

In [ ]:
untrained_prediction = model(tf.convert_to_tensor([sample]).numpy())
print('Logit-Werte des untrainierten Modells für das erste Bild:')
printValues(untrained_prediction, scale="logit", sample=sample)

Die **Softmax Funktion** für einen n-dimensionalen Vektor $z$ ist definiert als:
$$\sigma(z)_j=\frac{e^{z_i}}{\sum\limits_{i=0}^n e^{z_i}}$$

Die Werte der $\sigma(z)_j$ für jedes $j$ liegen dabei zwischen 0 und 1, und die Summe der $\sigma(z)_j$ addiert sich zu 1.

Der Wert kann daher als Wahrscheinlichkeit für die jeweilige Klasse $j$ interpretiert werden.

In [ ]:
print('Untrainierte Wahrscheinlichkeit für die Klassen bzgl des ersten Bildes:')
printValues(tf.nn.softmax(untrained_prediction).numpy(), sample=sample)

Der **SparseCategoricalCrossentropy**-Verlust nimmt einen Vektor von Logits und einen True-Index und gibt für jedes Sample den skalaren Verlust aus.

Dieser Verlust ist gleich der negativen Log-Wahrscheinlichkeit der wahren Klasse: Sie ist Null, wenn das Modell sicher ist, dass es die richtige Klasse hat.

Das verallgemeinert die Loss-Funktion, die wir aus der logistischen Regression kennen
$$-y log(\hat{y})-(1-y) log(1-\hat{y})$$
auf den Multi-Label Fall.

**Beachte:** SparseCategoricalCrossentropy ist für den Fall, wo die Multi-Label direkt über ihren Index angesprochen werden, CategoricalCrossentropy leistet dasselbe für One-hot encoded Labels, also statt `5` im ersten Fall, `[0, 0, 0, 0, 0, 1, 0,..., 0]` im letzteren.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Unser **untrainiertes Modell** gibt Wahrscheinlichkeiten nahe dem Zufallsprinzip an (1/10 für jede Klasse), so dass der Anfangsverlust nahe bei $-log(1/10) \approx 2.3$ liegen sollte.

In [ ]:
print(f'Für den True-Index {label} erhalten wir den Loss {loss_fn(label, untrained_prediction).numpy()} für das untrainierte Modell')

## Compilation des Modells

Der `compile`-Step konfiguriert das Modell für das Training: in diesem Schritt werden die Loss-Funktion, der Optimizer sowie eine auszuweisende Metrik festgelegt.

"adam" ist ein optimiertes Gradient Descent-Verfahren, wo die Learning Rate im Laufe der Iterationen angepasst wird.

*Beachte:* 'accuracy' im Zusammenspiel mit dem Loss `SparseCategoricalCrossentropy` ist `tf.keras.metrics.SparseCategoricalAccuracy` nicht `tf.metrics.Accuracy`!

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Training des Modells

Die Methode `Model.fit` passt die Modellparameter an, um den Verlust zu minimieren.

Es werden je Epoche Zwischenergebnisse ausgegeben. Eine **Epoche** ist ein Durchlauf durch die Trainingsdaten.

In [ ]:
epochs = 10

hist = model.fit(train, epochs=epochs, verbose=1);

Unter den Epochen zeigt Tensorflow die Anzahl der Iterationsschritte pro Epoche an. Da wir eine
Batch-Size von 64 festgelegt hatten, bekommen wir 938 Schritte pro Epoche: wir benötigen 938
Minibatch Schritte bei einer Batchsize von 64, um die 60.000 Fälle abzudecken.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,8))
x = np.arange(0, epochs)
ax[0].plot(x, hist.history['loss'], c='r')
ax[0].scatter(0,0, s=0.1)
ax[0].set_title('Loss', fontsize=18)
ax[0].set_xlabel('Epoche')
ax[1].plot(x, hist.history['accuracy'], c='g')
ax[1].scatter(0,0, s=0.1)
ax[1].set_title('Accuracy', fontsize=18)
ax[1].set_xlabel('Epoche');


## Evaluation des Modells

Mit `Model.evaluate` überprüfen wir jetzt die Performanz des Modells mit unserem Test-Set:

In [ ]:
(loss, accuracy) = model.evaluate(test, verbose=1)
print(f'\nLoss: {loss:.4f}, Accuracy: {accuracy*100:.1f}%')

## Softmax

Wir könnten den Softmax-Layer auch direkt in unser Modell integrieren, um als Ausgabe direkt eine Wahrscheinlichkeit für die jeweilige Klasse zu erhalten:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
samples = ds_test_scaled.batch(1)
for idx, sample in enumerate(samples):
    if idx > 10:
        break    
    printValues(probability_model(sample), sample=sample[0].numpy())
